In [1]:
# 1. ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x000002686207FD70>
# - https://discuss.huggingface.co/t/pretrain-model-not-accepting-optimizer/76209/20

!pip install --upgrade transformers
!pip install tf-keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/dataset_dict.json
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/validation/state.json
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/validation/dataset_info.json
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/validation/data-00000-of-00001.arrow
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/test/state.json
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/test/dataset_info.json
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/test/data-00000-of-00001.arrow
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/train/state.json
/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/train/dataset_info.json
/kaggle/input/tokenized-quo

In [3]:
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification
from datasets import load_from_disk

2024-07-27 04:54:56.978452: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 04:54:56.978576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 04:54:57.127786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
tokenized_dataset = load_from_disk(r"/kaggle/input/tokenized-quora-duplicates-dataset/tokenized_quora_duplicates_dataset/")
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['questions', 'is_duplicate', 'sentences1', 'sentences2', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 283003
    })
    test: Dataset({
        features: ['questions', 'is_duplicate', 'sentences1', 'sentences2', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 60644
    })
    validation: Dataset({
        features: ['questions', 'is_duplicate', 'sentences1', 'sentences2', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 60643
    })
})

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["is_duplicate"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_dataset["validation"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["is_duplicate"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/opt/conda/lib/python3.10/site-packages/datasets/arrow_dataset.py:410: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [7]:
num_epochs = 5

num_training_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.0,
    decay_steps=num_training_steps,
)

In [8]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
opt = Adam(learning_rate=lr_scheduler)

model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=opt,
    metrics=["accuracy"],
)

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1722056202.030455     119 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35376/35376 [==============================] - 3501s 96ms/step - loss: 0.3498 - accuracy: 0.8403 - val_loss: 0.3004 - val_accuracy: 0.8679
Epoch 2/5
35376/35376 [==============================] - 3333s 94ms/step - loss: 0.2569 - accuracy: 0.8912 - val_loss: 0.2956 - val_accuracy: 0.8818
Epoch 3/5
35376/35376 [==============================] - 3352s 95ms/step - loss: 0.1789 - accuracy: 0.9291 - val_loss: 0.2793 - val_accuracy: 0.8898
Epoch 4/5
35376/35376 [==============================] - 3372s 95ms/step - loss: 0.1071 - accuracy: 0.9604 - val_loss: 0.3125 - val_accuracy: 0.8940
Epoch 5/5
35376/35376 [==============================] - 3375s 95ms/step - loss: 0.0536 - accuracy: 0.9813 - val_loss: 0.4196 - val_accuracy: 0.8945


In [10]:
model.save_pretrained("quora_duplicate_question_detector")